In [2]:
import cv2
from win32api import GetSystemMetrics
import numpy as np
import math
print("Width =", GetSystemMetrics(0))
print("Height =", GetSystemMetrics(1))

Width = 1920
Height = 1080


In [100]:
import cv2
from win32api import GetSystemMetrics
import numpy as np
import pandas as pd
import math

# 파일명 한글 허용 함수
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None

# 이미지 크기 조절 함수
def resizeImage(image):
    (h, w) = image.shape[:2]
    x = GetSystemMetrics(0) / w * 0.8
    y = GetSystemMetrics(1) / h * 0.8
    if x<y:
        image = cv2.resize(image, (0,0), fx=x, fy=x)
    else:
        image = cv2.resize(image, (0,0), fx=y, fy=y)
    return image

for picNum in range(1,31):
    image = imread("04_top/"+ str(picNum) +".bmp")
    image = resizeImage(image)

    cv2.putText(image, str(picNum), (100,100),cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 1)

    # 외곽 사각형 그리기
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_8U)
    _, binary = cv2.threshold(laplacian, 60, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21,21))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel,iterations = 5)
    contours,_ = cv2.findContours(binary, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    mask = np.zeros_like(gray)

    c = max(contours, key=cv2.contourArea)

    rect = cv2.minAreaRect(c)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(image,[box],0,(0,0,255),1)

    # 사각형 변의 길이 출력
    for i in range(4):
        if i < 3:
            a = box[i][0] - box[i+1][0]
            b = box[i][1] - box[i+1][1]
            x = (box[i][0] + box[i+1][0])/2
            y = (box[i][1] + box[i+1][1])/2
        else :
            a = box[i][0] - box[0][0]
            b = box[i][1] - box[0][1]
            x = (box[i][0] + box[0][0])/2
            y = (box[i][1] + box[0][1])/2
        d = math.sqrt((a**2)+(b**2))
        cv2.putText(image, str(round(d, 2)), (int(x)+5,int(y)-10),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)

    # 사각형 꼭지점 좌표 출력
    for i in range(4):
        x = box[i][0]
        y = box[i][1]
        cv2.circle(image, (x, y),3, (0, 255, 255), 2)
        cv2.putText(image, str(x) + ", " + str(y), (x-5,y-10),cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1)

    # 원 그리기 함수
    def drawCircles(circles):
        for i in circles[0]:
            x = i[0]
            y = i[1]
            r = i[2]
            cv2.circle(image, (x, y), r, (255, 127, 0), 1)
            cv2.circle(mask, (x, y), r, (255, 255, 255), -1)
            if r <120:
                cv2.circle(image, (x, y), 3, (255, 255, 0), 1)
                cv2.putText(image, str(x) + ", " + str(y), (int(x)+5, int(y)-10),cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1)

    # 원 찾기 함수
    def findCircles(circles, minRadius, maxRadius):
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 800, param1 = 250, param2 = 10, minRadius = minRadius, maxRadius = maxRadius) 
        return circles

    # 찾을 원 범위 설정용 리스트
    circleList = [[50,95], [188,203], [220,248], [250,260]]
    if picNum == 5 or picNum == 6 or picNum == 21:
        circleList[1][1] = 208

    circles=[] # 찾은 원의 정보 저장용 변수
    r = [] # 원들의 반지름 저장용 변수

    # 원 찾고 그리기  - 4개(작은 원 부터 시작)
    for i, j in circleList:
        circles = findCircles(circles, i, j)
        r.append(circles[0][0])
        drawCircles(circles)

    mask = cv2.bitwise_and(mask, gray)# 가장 큰 원 추출

    # 원 안 컨투어 잡기
    _, binary2 = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    binary2 = cv2.morphologyEx(binary2, cv2.MORPH_CLOSE, kernel2,iterations = 2)
    contours2,_ = cv2.findContours(binary2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    c2 = max(contours2, key=cv2.contourArea)
    cv2.drawContours(image,[c2],0,(0,0,255),1)

    pr = int((r[1][2]+r[2][2])/2) - 5 # 1 2 원 사이의 원 반지름

    pt1 = cv2.ellipse2Poly((r[0][0], r[0][1]), (pr, pr), 0,0,360,1) # 1도 간격으로 원 위 점들의 좌표
    cv2.polylines(image, [pt1], False, (255,255,0), 1) # 점들로 원 그리기

    # 큰 원 안에 형상을 찾은 컨투어와 그린 원의 교차점과 근접한 점들을 찾음
    crossPoints = []
    for i in range(len(c2)):
        for j in range(len(pt1)):
            a = pt1[j][0] - c2[i][0][0]
            b = pt1[j][1] - c2[i][0][1]
            d = math.sqrt((a**2)+(b**2))
            if d <= 2 :
                crossPoints.append(c2[i][0])

    # 찾은 근접점 그리기
    for k in range(len(crossPoints)):
        cv2.circle(image, (crossPoints[k][0], crossPoints[k][1]),3, (255, 0, 255), 2)

    crossPointsDF = pd.DataFrame(crossPoints, columns=['x','y'])

    # 가까운 점들끼리 8개로 나눔
    cp = []
    for i in range(8):
        cp.append(crossPointsDF[(crossPointsDF.x < crossPointsDF.x.iloc[0]+10)&(crossPointsDF.x > crossPointsDF.x.iloc[0]-10)&(crossPointsDF.y < crossPointsDF.y.iloc[0]+10)&(crossPointsDF.y > crossPointsDF.y.iloc[0]-10)])
        for j in cp[i].index:
            crossPointsDF = crossPointsDF.drop([j])

    # 나눈 점들의 평균을 구함
    for i in range(8):
        cp[i] = cp[i].mean()

    # 평균점들끼리의 거리가 그린 원 지름과 비슷한 점들끼리 선을 그림
    for i in range(len(cp)):
        for j in range(len(cp)):
            x1 = int(cp[i][0])
            y1 = int(cp[i][1])
            x2 = int(cp[j][0])
            y2 = int(cp[j][1])
            a = x1 - x2
            b = y1 - y2
            d = math.sqrt((a**2)+(b**2))
            if d > pr*2-10:
                cv2.line(image, (x1, y1), (x2, y2), (0,255,255), 1)

    cv2.imshow('image', image)
    if cv2.waitKey(0) & 0xFF == 27:
        cv2.destroyAllWindows()
        break
    else:
        cv2.destroyAllWindows()

In [73]:
import cv2
from win32api import GetSystemMetrics
import numpy as np
import pandas as pd
# 파일명 한글 허용 함수
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None

# 이미지 크기 조절 함수
def resizeImage(image):
    (h, w) = image.shape[:2]
    x = GetSystemMetrics(0) / w * 0.8
    y = GetSystemMetrics(1) / h * 0.8
    if x<y:
        image = cv2.resize(image, (0,0), fx=x, fy=x)
    else:
        image = cv2.resize(image, (0,0), fx=y, fy=y)
    return image

# for picNum in range(1,31):
image = imread("04_top/3.bmp")
image = resizeImage(image)

# cv2.putText(image, str(picNum), (100,100),cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 1)

# 외곽 사각형 그리기
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

laplacian = cv2.Laplacian(gray, cv2.CV_8U)
_, binary = cv2.threshold(laplacian, 55, 255, cv2.THRESH_BINARY)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21,21))
binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel,iterations = 5)
contours,_ = cv2.findContours(binary, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
mask = np.zeros_like(gray)

c = max(contours, key=cv2.contourArea)

rect = cv2.minAreaRect(c)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(image,[box],0,(0,0,255),1)

# print(box)

# 사각형 변의 길이 출력
for i in range(4):
    if i < 3:
        a = box[i][0] - box[i+1][0]
        b = box[i][1] - box[i+1][1]
        x = (box[i][0] + box[i+1][0])/2
        y = (box[i][1] + box[i+1][1])/2
    else :
        a = box[i][0] - box[0][0]
        b = box[i][1] - box[0][1]
        x = (box[i][0] + box[0][0])/2
        y = (box[i][1] + box[0][1])/2
    d = math.sqrt((a**2)+(b**2))
    cv2.putText(image, str(round(d, 2)), (int(x)+5,int(y)-10),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)

# 사각형 꼭지점 좌표 출력
for i in range(4):
    x = box[i][0]
    y = box[i][1]
    cv2.circle(image, (x, y),3, (0, 255, 255), 2)
    cv2.putText(image, str(x) + ", " + str(y), (x-5,y-10),cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1)

# 원 그리기 함수
def drawCircles(circles):
    for i in circles[0]:
        x = i[0]
        y = i[1]
        r = i[2]
        cv2.circle(image, (x, y), r, (255, 127, 0), 1)
        cv2.circle(mask, (x, y), r, (255, 255, 255), -1)
        if r <120:
            cv2.circle(image, (x, y), 3, (255, 255, 0), 1)
            cv2.putText(image, str(x) + ", " + str(y), (int(x)+5, int(y)-10),cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1)

# 원 찾기 함수
def findCircles(circles, minRadius, maxRadius):
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 800, param1 = 250, param2 = 10, minRadius = minRadius, maxRadius = maxRadius) 
    return circles

# 찾을 원 범위 설정용 리스트
circleList = [[50,95], [190,203], [220,248], [250,260]]
# if picNum == 5 or picNum == 6 or picNum == 21:
#     circleList[1][1] = 208

circles=[] # 찾은 원의 정보 저장용 변수
r = [] # 원들의 반지름 저장용 변수

# 원 찾고 그리기  - 4개(작은 원 부터 시작)
for i, j in circleList:
    circles = findCircles(circles, i, j)
    r.append(circles[0][0])
    drawCircles(circles)

# print(circles)
# print(r)

# mask = cv2.bitwise_and(mask, gray)# 가장 큰 원 추출

# # 원 안 컨투어 잡기
# _, binary2 = cv2.threshold(mask, 75, 255, cv2.THRESH_BINARY)
# kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
# binary2 = cv2.morphologyEx(binary2, cv2.MORPH_CLOSE, kernel2,iterations = 1)
# contours2,_ = cv2.findContours(binary2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
# c2 = max(contours2, key=cv2.contourArea)
# cv2.drawContours(image,[c2],0,(0,0,255),1)

# pr = int((r[1][2]+r[2][2])/2) - 5 # 1 2 원 사이의 원 반지름

# pt1 = cv2.ellipse2Poly((r[0][0], r[0][1]), (pr, pr), 0,0,360,1) # 1도 간격으로 원 위 점들의 좌표
# cv2.polylines(image, [pt1], False, (255,255,0), 1) # 점들로 원 그리기

# # 큰 원 안에 형상을 찾은 컨투어와 그린 원의 교차점과 근접한 점들을 찾음
# crossPoints = []
# for i in range(len(c2)):
#     for j in range(len(pt1)):
#         a = pt1[j][0] - c2[i][0][0]
#         b = pt1[j][1] - c2[i][0][1]
#         d = math.sqrt((a**2)+(b**2))
#         if d <= 2 :
#             crossPoints.append(c2[i][0])

# # 찾은 근접점 그리기
# for k in range(len(crossPoints)):
#     cv2.circle(image, (crossPoints[k][0], crossPoints[k][1]),3, (255, 0, 255), 1)

# crossPointsDF = pd.DataFrame(crossPoints, columns=['x','y'])

# # 가까운 점들끼리 8개로 나눔
# cp = []
# for i in range(8):
#     cp.append(crossPointsDF[(crossPointsDF.x < crossPointsDF.x.iloc[0]+10)&(crossPointsDF.x > crossPointsDF.x.iloc[0]-10)&(crossPointsDF.y < crossPointsDF.y.iloc[0]+10)&(crossPointsDF.y > crossPointsDF.y.iloc[0]-10)])
#     for j in cp[i].index:
#         crossPointsDF = crossPointsDF.drop([j])

# # 나눈 점들의 평균을 구함
# for i in range(8):
#     cp[i] = cp[i].mean()

# # 평균점들끼리의 거리가 그린 원 지름과 비슷한 점들끼리 선을 그림
# for i in range(len(cp)):
#     for j in range(len(cp)):
#         x1 = int(cp[i][0])
#         y1 = int(cp[i][1])
#         x2 = int(cp[j][0])
#         y2 = int(cp[j][1])
#         a = x1 - x2
#         b = y1 - y2
#         d = math.sqrt((a**2)+(b**2))
#         if d > pr*2-10:
#             cv2.line(image, (x1, y1), (x2, y2), (0,255,255), 1)

cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# if cv2.waitKey(0) & 0xFF == 27:
#     cv2.destroyAllWindows()
#     break
# else:
#     cv2.destroyAllWindows()